# Task 01

In [1]:
class XPReplay():
    """"""""
    def __init__(self,size=1e5):
        self.max_size = size
        self.buffer = list()
        
    def store(self,xp):
        if len(self.buffer) < self.max_size:
            self.buffer.append(xp)
        else:
            self.buffer = self.buffer[1:]
            self.buffer.append(xp)

# Task 02

In [12]:
import gym
import tensorflow as tf
from tensorflow.keras import Model
import numpy as np

In [3]:
env = gym.make("LunarLander-v2").env

print("Observation space: {}".format(env.observation_space))
# There are 8 states: the coordinates of the lander in x & y, its linear velocities in x & y, its angle, 
# its angular velocity, and two booleans that represent whether each leg is in contact with the ground or not.

print()

print("Action space: {}".format(env.action_space))
# There are four discrete actions available: do nothing, fire left orientation engine, fire main engine, 
# fire right orientation engine


Observation space: Box([-1.5       -1.5       -5.        -5.        -3.1415927 -5.
 -0.        -0.       ], [1.5       1.5       5.        5.        3.1415927 5.        1.
 1.       ], (8,), float32)

Action space: Discrete(4)


/home/student/miniconda3/envs/gym/lib/python3.9/site-packages/gym/utils/passive_env_checker.py:97: UserWarning: WARN: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  logger.warn(


In [4]:
print(env.observation_space.low.size)
print(env.action_space.n)

8
4


In [5]:
class ANN(Model):
    def __init__(self,env):
        super(ANN,self).__init__()
        self.hidden_layer1 = tf.keras.layers.Dense(units=(env.observation_space.low.size)**2,
                                                  activation=tf.keras.activations.relu 
                                                  )
        self.hidden_layer2 = tf.keras.layers.Dense(units=(env.observation_space.low.size)**2,
                                                  activation=tf.keras.activations.relu
                                                  )
        self.output_layer = tf.keras.layers.Dense(units=env.action_space.n,
                                                  activation=tf.keras.activations.linear
                                                  )
    def call(self,x):
        x = self.hidden_layer1(x)
        x = self.hidden_layer2(x)
        x = self.output_layer(x)
        return x

In [8]:
env = gym.make("LunarLander-v2").env
ann = ANN(env)
input = tf.reshape(env.reset(),shape=(1,-1))
print(input)
ann.call(input)

tf.Tensor(
[[-0.00512962  1.4200025  -0.51958984  0.40364882  0.00595074  0.11769489
   0.          0.        ]], shape=(1, 8), dtype=float32)


<tf.Tensor: shape=(1, 4), dtype=float32, numpy=
array([[ 0.12855735, -0.12689298, -0.04397256,  0.02603916]],
      dtype=float32)>

# Task 03

In [15]:
# initialize environment
env = gym.make("LunarLander-v2").env
# initialize ANN
ann = ANN(env)
# create empty replay buffer
replay =  XPReplay()

# hyper-parameters
# I'm making these up with no reference. They're probably trash
n_episodes = 1
n_train_step = 1
batch_size = 2

# play n episodes
for i in range(n_episodes):
    s = env.reset()
    done = False
    # until the episode is finished, sample s,a,r,s' and add them to the replay buffer
    while done == False:
        q = ann.call(tf.reshape(s,shape=(1,-1)))
        # implement epsilon greedy here later
        a = np.argmax(q)
        s_new, r, done, _ = env.step(a)
        sars = np.asarray((s,a,r,s_new),dtype=object)
        replay.store(sars)
        s = s_new
    #print(replay.buffer[0])
    # train the newtwork
    # perform n training steps
    for j in range(n_train_step):
        # sample a minibatch of s,a,r,s' from replay buffer
        samples = random.sample(replay.buffer,batch_size)
        



[array([-0.00627804,  1.4129663 , -0.6359073 ,  0.09091699,  0.00728139,
         0.14404237,  0.        ,  0.        ], dtype=float32)           2
 -2.358887885803301
 array([-0.01261435,  1.4158394 , -0.6405796 ,  0.12765545,  0.01415315,
         0.13744836,  0.        ,  0.        ], dtype=float32)          ]
